In [1]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, TrainingArguments, Trainer
import torch

fe = AutoFeatureExtractor.from_pretrained("microsoft/wavlm-base")
model = AutoModelForAudioClassification.from_pretrained("microsoft/wavlm-base", num_labels=29)

2024-04-25 04:39:39.582480: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 04:39:39.582541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 04:39:39.588651: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

In [ ]:
fe

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# create an example list of labels
import pickle
import numpy as np
with open('/kaggle/working/data.pkl', 'rb') as file:
    listdata = pickle.load(file)
x=[]
y=[]
for data in listdata:
    x.append(data['audio'])
    y.append(data['speaker'])

labels = y
# create a LabelEncoder object
le = LabelEncoder()
# fit the encoder to the labels and transform the labels
y = le.fit_transform(labels)
print(len(le.classes_))
# print the original labels and the encoded labels
print("Encoded labels:", y)
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)
# Save the encoded data as a NumPy array
np.save('encoded_data.npy', y)

29
Encoded labels: [ 1  1  1 ... 28 28 28]


In [3]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
from datasets import load_dataset
dataset = load_dataset('common_language', split = 'test')

In [ ]:
# def audio_extractor(example):
#   return fe(dataset["audio"]["array"], sampling_rate=fe.sampling_rate, return_tensors="pt")

In [ ]:
!pip install gdown

In [ ]:
import gdown

file_id = "your_file_id"
url = f""

output = "data.pkl"  # Specify the name of the downloaded file

gdown.download(url, output, quiet=False)

In [4]:
import datasets
dataset_train = datasets.Dataset.from_dict({
  'audio' :x_train,
  'label':y_train
})
dataset_test = datasets.Dataset.from_dict({
  'audio' :x_test,
  'label':y_test
})

In [5]:
dataset_train

Dataset({
    features: ['audio', 'label'],
    num_rows: 9594
})

In [ ]:
import datasets
dataset = dataset.cast_column(
        'audio', datasets.features.Audio(sampling_rate=fe.sampling_rate)
    )

In [ ]:
torch.Long

In [6]:
import torch
def collator_rel(features):
    batch = {}
    batch["labels"] = torch.tensor([f["label"] for f in features], dtype=int)
    encodings = fe([f['audio'] for f in features], sampling_rate=fe.sampling_rate, return_tensors="pt", padding='max_length', truncation=True, max_length=40000 )
    batch['input_values'] = encodings['input_values']
    batch['attention_mask'] = encodings['attention_mask']
    return batch

In [ ]:
def collator_fr(features):
  batch = {}
  batch["labels"] = torch.tensor([f["language"] for f in features], dtype=int)
  encodings = fe([f['audio']['array'] for f in features], sampling_rate=fe.sampling_rate, return_tensors="pt", padding='max_length', truncation=True, max_length=80000 )
  batch['input_values'] = encodings['input_values']
  batch['attention_mask'] = encodings['attention_mask']
  return batch
# coll = collator_fr([dataset[i] for i in range(6)])


In [ ]:
!pip install transformers[torch]

In [7]:

training_args = TrainingArguments("test-trainer", remove_unused_columns = False, num_train_epochs=4, logging_steps=400, per_device_train_batch_size=4, evaluation_strategy="steps", eval_steps=400)

In [8]:
print(training_args)

TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=400,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_l

In [9]:
from sklearn.metrics import accuracy_score
from transformers import Trainer
def compute_accuracy(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {"accuracy": accuracy_score(labels, preds)}

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset_train,
    eval_dataset = dataset_test,
    data_collator=collator_rel,
    compute_metrics=compute_accuracy,)
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: duymaxvip (duysop). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
400,2.670600,1.902777,0.429718
800,1.579100,1.123005,0.697957
1200,0.910200,0.481851,0.912695
1600,0.503700,0.361022,0.912938
2000,0.298600,0.252571,0.934582
2400,0.254300,0.190562,0.950632
2800,0.217900,0.206433,0.959144
3200,0.204800,0.175900,0.962792
3600,0.158400,0.149914,0.972519
4000,0.114200,0.268620,0.955982


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/fun

TrainOutput(global_step=4800, training_loss=0.5962532575925191, metrics={'train_runtime': 4090.9009, 'train_samples_per_second': 9.381, 'train_steps_per_second': 1.173, 'total_flos': 8.7112672350912e+17, 'train_loss': 0.5962532575925191, 'epoch': 4.0})

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=fe)

In [ ]:
data_collator

In [ ]:
dataset['path'][0]

In [ ]:
from transformers import AutoProcessor, WavLMModel
import torch
from datasets import load_dataset

# dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
# dataset = dataset.sort("id")
# sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")
# model = WavLMModel.from_pretrained("patrickvonplaten/wavlm-libri-clean-100h-base-plus")

# # audio file is decoded on the fly
# inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
# with torch.no_grad():
#     outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state
# list(last_hidden_states.shape)


In [ ]:
processor

In [ ]:
inputs = processor([dataset[2]["audio"]["array"], dataset[2]["audio"]["array"]], sampling_rate=sampling_rate, return_tensors="pt")

In [ ]:
inputs

In [ ]:
inputs['attention_mask'].shape

In [10]:
import zipfile
import os

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

folder_to_zip = '/kaggle/working/test-trainer/checkpoint-3500'
output_zip_file = 'Wav30fr.zip'

zip_folder(folder_to_zip, output_zip_file)

In [ ]:
import numpy as np
import pickle
inputs=['số 50 ngõ 87 Lê Thanh Nghị, Đống Đa nhé bạn']
encodeing = tokenizer(inputs)
with torch.no_grad():
  outputs = model(**encodeing)
  predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)